In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import pickle

In [ ]:
from matplotlib.ticker import StrMethodFormatter

In [ ]:
import sys
sys.path.append("../")

In [ ]:
from alpha_rank import alpha_rank
from functools import partial
import os

In [ ]:
from scipy.stats import kendalltau
from sampling_test import _kendall_partial as kendall_partial
from scipy.stats import entropy

In [ ]:
# Name of the folder you want to save graphs too
exps_name = "Graphs"

# Only load the runs with this experiment label
labels = []

# Directory where the runs are saved
direc_to_look = "..."

# Only look in these folders in that directory
folders_to_look = []

In [ ]:
exps_name

In [ ]:
labels

In [ ]:
folders_to_look

In [ ]:
direc_to_look

In [ ]:
direc_to_save = "" # Where you want to save the stuff
orignal_direc = "{}/{}".format(direc_to_save, exps_name)
os.makedirs(orignal_direc, exist_ok=True)
os.makedirs(orignal_direc+"/median", exist_ok=True)
os.makedirs(orignal_direc+"/mean", exist_ok=True)

In [ ]:
# Split the data on these parameters
params = [
            "sampler",
            "acquisition",
            "delta",
            "starting_var",
            "noise_var",
            ]

# Seperate graphs for these
facets = [
    "env"
]

In [ ]:
def filter_func(exp_info):
    if exp_info["label"] not in labels:
        return False

    return True

In [ ]:
all_data = {}
keys_no_repeat = set()
all_faces = set()

files_to_check = glob.glob("{}/{}/*.pkl".format(direc_to_look, "*"))
print("TOTAL FILES TO CHECK:", len(files_to_check))
for i, d in enumerate(files_to_check):
    skip_entry = not any([fs in d for fs in folders_to_look])
    if skip_entry:
        continue
    print(i,end=",")
    with open(d, "rb") as f:
        data_dict = pickle.load(f)
        exp_info = data_dict["exp_info"]
        if not filter_func(exp_info):
            continue
        params_str = "__".join([str(exp_info.get(s, "N/A")) for s in params])
        keys_no_repeat.add(params_str)
        params_str += "#{}_{}_{}".format(exp_info["repeats"], labels.index(exp_info["label"]), [f in d for f in folders_to_look].index(True))
        faces_str = "__".join([str(exp_info.get(s, "N/A")) for s in facets])
        all_faces.add(faces_str)
        full_str = "{}++{}".format(faces_str, params_str)
        runs = all_data.get(full_str, [])
        runs.append(data_dict)
        all_data[full_str] = runs
        
print("\nDONE!")

In [ ]:
# Function to filter what runs are graphed (from amongst the runs loaded)
def filter_func2(exp_info):
    
    # For instance if we didn't want to graph the lines for ResponseGraphUCB:
    #     if exp_info["sampler"] == "freq2":
    #         return False

    # Return True if you want the line
    return True

In [ ]:
# Load the data and do some more filtering if you want
data = {}
faces = set()
runs_per = {}

for k in all_data.keys():
    exp_info = all_data[k][0]["exp_info"]
    if not filter_func2(exp_info):
        continue
    faces_str = "__".join([str(exp_info.get(s, "N/A")) for s in facets])
    faces.add(faces_str)
    data[k] = all_data[k]
    no_r_key = k.split("#")[0]
    val = runs_per.get(no_r_key, 0)
    runs_per[no_r_key] = val + 1

In [ ]:
print("Keys:",keys_no_repeat,"\n")
print("Keys Size:",len(keys_no_repeat),"\n")
print("Length:",len(data.values()),"\n")
print("Faces:", faces, "\n")
data_keys = list(data.keys())

print("Faces: [{}]".format(len(faces)))
for face in faces:
    print(face)
    
print("\nRepeats per:")
for k in sorted(runs_per.keys()):
    print("{}: [{}]".format(k, runs_per[k]))

print("\n------\n")
for k in data.keys():
    a = data[k]
    print(k, ":", len(a))    

In [ ]:
# Get data common to all the runs
run_data = data[data_keys[0]][0]

exp = run_data["exp_info"]
t_max = exp["t_max"]
num_pops = run_data["env_info"]["num_pops"]
num_players = run_data["env_info"]["num_players"]
num_strats = run_data["env_info"]["num_strats"]

true_payoff = run_data["env_info"]["true_payoffs"]
alpha_rank_partial = partial(alpha_rank, alpha=exp["alpha"], mutation=exp["alpha_mutation"], use_inf_alpha=exp["inf_alpha"], inf_alpha_eps=exp["inf_alpha_eps"])
true_alpha_rank = alpha_rank_partial(true_payoff)
alpha_rank_strats = true_alpha_rank.shape[0]

# For every run we might have a seperate environment, fill in the data about the true alpha ranking
for key in data_keys:
    for run in data[key]:
        run_payoff = run["env_info"]["true_payoffs"]
        true_alpha_rank = alpha_rank_partial(run_payoff)
        run["env_info"]["true_alpha_rank"] = true_alpha_rank

In [ ]:
# If ResponseGraphUCB has stopped sampling, duplicate the entries at the end to match up with t_max
for key in data.keys():
    for runs in data[key]:
        if len(runs["mean_alpha_rankings"]) < t_max:
            old = runs["mean_alpha_rankings"]
            new_data = old + [old[-1] for _ in range(t_max - len(old))]
            runs["mean_alpha_rankings"] = new_data
        if len(runs["alpha_rankings_more"]) < 100:
            print(key,end=",")
            old = runs["alpha_rankings_more"]
            new_data = old + [old[-1] for _ in range(100 - len(old))]
            runs["alpha_rankings_more"] = new_data

In [ ]:
max_repeats = 20

In [ ]:
keys_to_plot = list(runs_per.keys())

In [ ]:
def smooth_ma(y, box_pts):
    box = np.ones(box_pts)/box_pts
    s=np.concatenate([np.array([y[0] for _ in range(box_pts//2)]), y, np.array([y[-1] for _ in range(box_pts//2)])])
    y_smooth = np.convolve(s, box, mode='valid')
    return y_smooth

In [ ]:
# Convert the names of the runs into something nice/readable
def get_nice_name(k):
    if "++bayesian__ndd" in k:
        return r"$\alpha$IG (NSB)"
    if "++bayesian__entropy" in k:
        return r"$\alpha$IG (Bins)"
    if "++bayesian__l1" in k:
        return r"$\alpha$Wass"
    if "++freq2" in k:
        return "RG-UCB"
    if "++random" in k:
        return "Uniform"
    if "++payoff" in k:
        return "Payoff"

In [ ]:
# Function to graph data with the style used in the paper
def graph(  keys,
            processer, 
            aggregate=lambda x: np.mean(x, axis=0), 
            stat=lambda x: np.mean(x, axis=0),
            upper=lambda x: np.mean(x, axis=0) + np.std(x, axis=0),
            lower=lambda x: np.mean(x, axis=0) - np.std(x, axis=0),
            smooth=True,
            smooth_window=101,
            algo_colours=True):

    keys_r_data = {}
    
    keys_data = []
    keys_upper = []
    keys_lower = []
    keys_name = []
    keys_num = []
    
    print("Getting quantities.")
    for key in keys:
        print(key, end=", ")
        key_data = data[key] # List of dictionarties
        processed_data = []
        for run in key_data:
            points = processer(run)
            processed_data.append(points)
        processed_data = np.array(processed_data)
        aggregated_data = aggregate(processed_data) # For each repeat we have the data we want to graph
        
        no_r_key = key.split("#")[0]
        if no_r_key not in keys_r_data:
            keys_r_data[no_r_key] = [aggregated_data]
        else:
            keys_r_data[no_r_key].append(aggregated_data)
        
    print("Computing stats")
    for key in keys_r_data.keys():
        concat_data = np.stack(keys_r_data[key], axis=0)[:max_repeats]
        
        s_data = stat(concat_data)
        u_data = upper(concat_data)
        l_data = lower(concat_data)

        keys_data.append(s_data)
        keys_upper.append(u_data)
        keys_lower.append(l_data)
        keys_name.append(key)
        keys_num.append(len(concat_data))
    
    colour_scheme = {
        "bayesian__l1": "#d8345f",
        "bayesian__ndd": "#6886c5",
        "bayesian__entropy": "#b19cd9",
        "freq2": "#649d66",
        "random": "#656569",
        "payoff": "#ec823a",
    }

    print("Plotting")
    plt.figure(figsize=(12,10))
    for name, x, xu, xl, n in zip(keys_name, keys_data, keys_upper, keys_lower, keys_num):
        xs = range(len(x))
        smoothed_x = smooth_ma(x, smooth_window) if smooth else x
        color_to_use = None
        for cks in colour_scheme.keys():
            if cks in name:
                color_to_use = colour_scheme[cks]
        if algo_colours:
            label_name = get_nice_name(name)
            plt.plot(xs, smoothed_x, label="{} [{}]".format(label_name, n), linewidth=3, alpha=0.9, color=color_to_use)
            plt.fill_between(xs, xl if not smooth else smooth_ma(xl, smooth_window), xu if not smooth else smooth_ma(xu, smooth_window), alpha=0.3, color=color_to_use)
        else:
            plt.plot(xs, smoothed_x, label="{} [{}]".format(name, n), linewidth=3, alpha=0.9)
            plt.fill_between(xs, xl if not smooth else smooth_ma(xl, smooth_window), xu if not smooth else smooth_ma(xu, smooth_window), alpha=0.3)
    if algo_colours:
        handles, labels = plt.gca().get_legend_handles_labels()
        name_order = [r"$\alpha$IG (Bins)", r"$\alpha$IG (NSB)", r"$\alpha$Wass", "RG-UCB" ,"Payoff", "Uniform"]
        if len(labels) < len(name_order):
            # We're not graphing Uniform, leave it out of the name_order
            name_order = name_order[:-1]
        order = [[i for i,l in enumerate(labels) if l.startswith(n)][0] for n in name_order]
        leg = plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order], bbox_to_anchor=(1.01, 1), loc=2, fontsize=36, handlelength=1)
    else:
        leg = plt.legend(bbox_to_anchor=(1.05, 1), loc=2)

    for legobj in leg.legendHandles:
        legobj.set_linewidth(4 if not algo_colours else 10)
        
    plt.gca().xaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))

In [ ]:
SAVING = True

In [ ]:
# Median func defaults
med_f = lambda x: np.median(x, axis=0)
med_l = lambda x: np.min(x, axis=0)
med_u = lambda x: np.max(x, axis=0)
median_graph = partial(graph, stat=med_f, lower=med_l, upper=med_u)

In [ ]:
direc = "{}/median".format(orignal_direc)
def save_plot(name, face, m=None):
    os.makedirs(direc+"/{}".format(face), exist_ok=True)
    if m is None:
        plt.savefig("{}/{}/{}.png".format(direc, face, name), bbox_inches='tight')
    else:
        plt.savefig("{}/{}/{}__{}.png".format(direc, face, name, m), bbox_inches='tight')

In [ ]:
title_to_use = "3 Good, 5 Bad"

In [ ]:
use_nice_name = True

In [ ]:
for face in faces:
    print("Face: {}".format(face))
    if "random__" in face:
        continue
    keys = set([k for k in data.keys() if k.startswith("{}++".format(face)) and k.split("#")[0] in keys_to_plot and "random" not in k])
    single_env = True if len(data[list(keys)[0]]) == 1 else False
    print("Single env:", single_env)
    
    # Frequentist regret = 1- P(Correct alpha_rank)
    for cc in [0.01]:
        def p_alpha_correct(d):
            return [1 - np.mean([np.abs(np.around(a, 3)-np.around(d["env_info"]["true_alpha_rank"], 3)).sum()<cc for a in x]) for x in d["alpha_rankings_more"]]
        median_graph(keys, p_alpha_correct, smooth=False, smooth_window=101,algo_colours=use_nice_name,
                        stat = lambda x: np.mean(x, axis=0),
                        lower = lambda x: np.mean(x, axis=0) - np.std(x, axis=0)/np.sqrt(x.shape[0]),
                        upper = lambda x: np.mean(x, axis=0) + np.std(x, axis=0)/np.sqrt(x.shape[0]),
                        use_parallel=True)

        plt.xlabel("Samples", fontsize=40)
        plt.ylabel(r"$1 - P(r_{GT})$", fontsize=40)
        plt.ylim(-0.05,1.05)
        plt.xticks([0,20,40,60,80,100], ["{:,.0f}".format(x * t_max) for x in [0,0.2,0.4,0.6,0.8,1.0]], fontsize=34)
        plt.yticks(fontsize=34)
        plt.title(title_to_use, fontsize=40)
        plt.legend('',frameon=False)
        if SAVING:
            save_plot("freq_regret_{}".format(cc), face)
        plt.close()
        print("Plotted freq regret,",cc)

In [ ]:
for face in faces:
    print("Face: {}".format(face))
    if "random__" in face:
        continue
    keys = set([k for k in data.keys() if k.startswith("{}++".format(face)) and k.split("#")[0] in keys_to_plot  and "random" not in k])
    single_env = True if len(data[list(keys)[0]]) == 1 else False
    print("Single env:", single_env)
        
    # Bayesian regret = P(Most probable alpha_rank)
    def prob_ps(d):
        pp = []
        for x in d["alpha_rankings_more"]:
            unique_phis, counts = np.unique(np.around(x, decimals=3), return_counts=True, axis=0)
            argmax_entry = np.argmax(counts)
            p_phi = unique_phis[argmax_entry]
            p_phi_prob = counts[argmax_entry] / sum(counts)
            pp.append(1 - p_phi_prob)
        return pp
    median_graph(keys, prob_ps,smooth=False, smooth_window=101,algo_colours=use_nice_name,
                        stat = lambda x: np.mean(x, axis=0),
                        lower = lambda x: np.mean(x, axis=0) - np.std(x, axis=0)/np.sqrt(x.shape[0]),
                        upper = lambda x: np.mean(x, axis=0) + np.std(x, axis=0)/np.sqrt(x.shape[0]))

    plt.xlabel("Samples", fontsize=40)
    plt.ylabel(r"$1 - P(r_{*})$", fontsize=40)
    plt.ylim(-0.05,1.05)
    plt.xticks([0,20,40,60,80,100], ["{:,.0f}".format(x * t_max) for x in [0,0.2,0.4,0.6,0.8,1.0]], fontsize=34)
    plt.yticks(fontsize=34)
    plt.title(title_to_use, fontsize=40)
    plt.legend('',frameon=False)
    if SAVING:
        save_plot("bayesian_regret", face)
    plt.close()
    print("Plotted bayesian_regret.")    

In [ ]:
for face in faces:
    print("Face: {}".format(face))
    keys = set([k for k in data.keys() if k.startswith("{}++".format(face)) and k.split("#")[0] in keys_to_plot])
    single_env = True if len(data[list(keys)[0]]) == 1 else False
    print("Single env:", single_env)
    
    # Number of seeds correct
    m_alpha_rank = lambda d: [x[0] for x in d["mean_alpha_rankings"]]
    alpha_correct = lambda d: [1- np.linalg.norm(a-d["env_info"]["true_alpha_rank"], ord=1)<0.01 for a in m_alpha_rank(d)]
    graph(keys, alpha_correct,
            stat = lambda x: np.mean(x, axis=0),
            lower = lambda x: np.mean(x, axis=0) - np.std(x, axis=0)/np.sqrt(x.shape[0]),
            upper = lambda x: np.mean(x, axis=0) + np.std(x, axis=0)/np.sqrt(x.shape[0]),
            smooth=True,
            smooth_window=401 if t_max >= 20*1000 else 201,
            algo_colours=use_nice_name)
    plt.xlabel("Samples", fontsize=40)
    plt.xticks(fontsize=34)
    plt.ylabel(r"$1 - P(f(\bar{M}) = r_{gt})$", fontsize=40)
    plt.yticks(fontsize=34)
    plt.ylim(-0.05,1.05)
    plt.title(title_to_use, fontsize=40)
    if SAVING:
        save_plot("mean_correct", face)
    plt.close()
    print("Plotted % correct alpha ranking.")